In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error

In [3]:
import mlflow

mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_experiment('nyc-taxi-experiment')

2024/05/31 18:19:19 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///c:/Users/daniel/Documents/Universidad/Cursos/MLOpsZoomcamp/02_experiment-tracking/mlruns/1', creation_time=1717201159692, experiment_id='1', last_update_time=1717201159692, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [4]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df['lpep_dropoff_datetime'] - df['lpep_pickup_datetime']
    df['duration'] = df.duration.apply(lambda x: x.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [6]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [7]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[numerical + categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[numerical + categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [9]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

7.758715203341164

In [10]:
import pickle

In [11]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [14]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'daniel')

    mlflow.log_param('train-data-path', '/.data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', '.data/green_tripdata_2021-02.parquet')


    alpha = 0.1

    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)


### Hyperopt

In [15]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [17]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [18]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [20]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=4,
    trials=Trials()
)

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [18:58:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:9.17653                          
[1]	validation-rmse:7.71768                          
[2]	validation-rmse:7.05823                          
[3]	validation-rmse:6.76452                          
[4]	validation-rmse:6.62646                          
[5]	validation-rmse:6.55771                          
[6]	validation-rmse:6.51470                          
[7]	validation-rmse:6.49049                          
[8]	validation-rmse:6.47520                          
[9]	validation-rmse:6.46036                          
[10]	validation-rmse:6.45278                         
[11]	validation-rmse:6.44720                         
[12]	validation-rmse:6.44165                         
[13]	validation-rmse:6.43614                         
[14]	validation-rmse:6.43298                         
[15]	validation-rmse:6.43167                         
[16]	validation-rmse:6.42795                         
[17]	validation-rmse:6.42547                         
[18]	validation-rmse:6.42340

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [18:59:25] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.26282                                                  
[1]	validation-rmse:10.46239                                                  
[2]	validation-rmse:9.79021                                                   
[3]	validation-rmse:9.22851                                                   
[4]	validation-rmse:8.76313                                                   
[5]	validation-rmse:8.37109                                                   
[6]	validation-rmse:8.04940                                                   
[7]	validation-rmse:7.78980                                                   
[8]	validation-rmse:7.57886                                                   
[9]	validation-rmse:7.39817                                                   
[10]	validation-rmse:7.25408                                                  
[11]	validation-rmse:7.13877                                                  
[12]	validation-rmse:7.04057                        

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [19:01:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.49657                                                  
[1]	validation-rmse:10.86454                                                  
[2]	validation-rmse:10.30884                                                  
[3]	validation-rmse:9.82194                                                   
[4]	validation-rmse:9.39657                                                   
[5]	validation-rmse:9.02644                                                   
[6]	validation-rmse:8.70530                                                   
[7]	validation-rmse:8.42730                                                   
[8]	validation-rmse:8.18733                                                   
[9]	validation-rmse:7.98128                                                   
[10]	validation-rmse:7.80372                                                  
[11]	validation-rmse:7.65004                                                  
[12]	validation-rmse:7.51860                        

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [19:02:14] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.44195                                                  
[1]	validation-rmse:10.76782                                                  
[2]	validation-rmse:10.18092                                                  
[3]	validation-rmse:9.67121                                                   
[4]	validation-rmse:9.23132                                                   
[5]	validation-rmse:8.85161                                                   
[6]	validation-rmse:8.52594                                                   
[7]	validation-rmse:8.24739                                                   
[8]	validation-rmse:8.00949                                                   
[9]	validation-rmse:7.80740                                                   
[10]	validation-rmse:7.63563                                                  
[11]	validation-rmse:7.48965                                                  
[12]	validation-rmse:7.36577                        

In [21]:
params = {
    'learning_rate': 0.39867389055557895,
    'max_depth': 78,
    'min_child_weight': 3.10034390329303,
    'objective': 'reg:linear',
    'reg_alpha': 0.2191404388930433,
    'reg_lambda': 0.04834588312304234,
    'seed': 42
}


mlflow.xgboost.autolog() # Esta línea activa la función de autologging de MLflow para XGBoost. Esto significa que MLflow registrará automáticamente los parámetros, las métricas y el modelo entrenado durante el entrenamiento del modelo XGBoost.

booster = xgb.train(
    params=params,
    dtrain=train,
    num_boost_round=100,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2024/05/31 19:46:54 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '1b2e9f17677e41c483f2f7ccc0519500', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow
c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [19:47:04] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.17653
[1]	validation-rmse:7.71768
[2]	validation-rmse:7.05823
[3]	validation-rmse:6.76452
[4]	validation-rmse:6.62646
[5]	validation-rmse:6.55771
[6]	validation-rmse:6.51470
[7]	validation-rmse:6.49049
[8]	validation-rmse:6.47520
[9]	validation-rmse:6.46036
[10]	validation-rmse:6.45278
[11]	validation-rmse:6.44720
[12]	validation-rmse:6.44165
[13]	validation-rmse:6.43614
[14]	validation-rmse:6.43298
[15]	validation-rmse:6.43167
[16]	validation-rmse:6.42795
[17]	validation-rmse:6.42547
[18]	validation-rmse:6.42340
[19]	validation-rmse:6.42129
[20]	validation-rmse:6.41880
[21]	validation-rmse:6.41626
[22]	validation-rmse:6.41378
[23]	validation-rmse:6.41241
[24]	validation-rmse:6.41130
[25]	validation-rmse:6.41017
[26]	validation-rmse:6.40904
[27]	validation-rmse:6.40497
[28]	validation-rmse:6.40331
[29]	validation-rmse:6.40188
[30]	validation-rmse:6.40065
[31]	validation-rmse:6.39880
[32]	validation-rmse:6.39802
[33]	validation-rmse:6.39718
[34]	validation-rmse:6.3

2024/05/31 19:48:38 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/31 19:48:38 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [19:48:38] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."


### Model management with MLflow

In [22]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'daniel')

    mlflow.log_param('train-data-path', '/.data/green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data-path', '.data/green_tripdata_2021-02.parquet')


    alpha = 0.1

    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)

    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle') # esta linea sube el modelo a mlflow

In [23]:
with mlflow.start_run():
    best_params = {
    'learning_rate': 0.39867389055557895,
    'max_depth': 78,
    'min_child_weight': 3.10034390329303,
    'objective': 'reg:linear',
    'reg_alpha': 0.2191404388930433,
    'reg_lambda': 0.04834588312304234,
    'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow') # esta linea sube el modelo a mlflow

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:41:45] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.17653
[1]	validation-rmse:7.71768
[2]	validation-rmse:7.05823
[3]	validation-rmse:6.76452
[4]	validation-rmse:6.62646
[5]	validation-rmse:6.55771
[6]	validation-rmse:6.51470
[7]	validation-rmse:6.49049
[8]	validation-rmse:6.47520
[9]	validation-rmse:6.46036
[10]	validation-rmse:6.45278
[11]	validation-rmse:6.44720
[12]	validation-rmse:6.44165
[13]	validation-rmse:6.43614
[14]	validation-rmse:6.43298
[15]	validation-rmse:6.43167
[16]	validation-rmse:6.42795
[17]	validation-rmse:6.42547
[18]	validation-rmse:6.42340
[19]	validation-rmse:6.42129
[20]	validation-rmse:6.41880
[21]	validation-rmse:6.41626
[22]	validation-rmse:6.41378
[23]	validation-rmse:6.41241
[24]	validation-rmse:6.41130
[25]	validation-rmse:6.41017
[26]	validation-rmse:6.40904
[27]	validation-rmse:6.40497
[28]	validation-rmse:6.40331
[29]	validation-rmse:6.40188
[30]	validation-rmse:6.40065
[31]	validation-rmse:6.39880
[32]	validation-rmse:6.39802
[33]	validation-rmse:6.39718
[34]	validation-rmse:6.3

2024/05/31 20:43:02 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2024/05/31 20:43:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:43:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified."
c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:43:07] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.c

In [24]:
mlflow.xgboost.autolog(disable=True) # Esta línea desactiva la función de autologging de MLflow para XGBoost. Esto significa que MLflow no registrará automáticamente los parámetros, las métricas y el modelo entrenado durante el entrenamiento del modelo XGBoost.

In [25]:
with mlflow.start_run():
    best_params = {
    'learning_rate': 0.39867389055557895,
    'max_depth': 78,
    'min_child_weight': 3.10034390329303,
    'objective': 'reg:linear',
    'reg_alpha': 0.2191404388930433,
    'reg_lambda': 0.04834588312304234,
    'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric('rmse', rmse)

    with open ('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out) # esta linea guarda el preprocesador

    mlflow.log_artifact(local_path='models/preprocessor.b', artifact_path='preprocessor') # esta linea sube el preprocesador a mlflow


    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow') # esta linea sube el modelo a mlflow


c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:49:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:9.17653
[1]	validation-rmse:7.71768
[2]	validation-rmse:7.05823
[3]	validation-rmse:6.76452
[4]	validation-rmse:6.62646
[5]	validation-rmse:6.55771
[6]	validation-rmse:6.51470
[7]	validation-rmse:6.49049
[8]	validation-rmse:6.47520
[9]	validation-rmse:6.46036
[10]	validation-rmse:6.45278
[11]	validation-rmse:6.44720
[12]	validation-rmse:6.44165
[13]	validation-rmse:6.43614
[14]	validation-rmse:6.43298
[15]	validation-rmse:6.43167
[16]	validation-rmse:6.42795
[17]	validation-rmse:6.42547
[18]	validation-rmse:6.42340
[19]	validation-rmse:6.42129
[20]	validation-rmse:6.41880
[21]	validation-rmse:6.41626
[22]	validation-rmse:6.41378
[23]	validation-rmse:6.41241
[24]	validation-rmse:6.41130
[25]	validation-rmse:6.41017
[26]	validation-rmse:6.40904
[27]	validation-rmse:6.40497
[28]	validation-rmse:6.40331
[29]	validation-rmse:6.40188
[30]	validation-rmse:6.40065
[31]	validation-rmse:6.39880
[32]	validation-rmse:6.39802
[33]	validation-rmse:6.39718
[34]	validation-rmse:6.3

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:50:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


### Cargar el modelo desde MLflow

In [26]:
logged_model = 'runs:/d46a9372875f43dc93bc417a5addfef2/models_mlflow' # esta linea guarda el modelo en una variable

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model) # esta linea carga el modelo como un PyFuncModel


c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:52:16] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [27]:
xgboost_model = mlflow.xgboost.load_model(logged_model) # esta linea carga el modelo como un XGBoostModel

c:\Users\daniel\Documents\Universidad\Cursos\MLOpsZoomcamp\.venv\lib\site-packages\xgboost\core.py:160: UserWarning: [20:53:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [28]:
y_pred = xgboost_model.predict(valid)

y_pred[:10]

array([13.944463 ,  7.2496753, 13.421872 , 24.997238 ,  9.630789 ,
       17.118036 , 10.254211 ,  8.295286 ,  9.202396 , 17.306215 ],
      dtype=float32)